<font color ='Navy'><center> __BruFace__ </center></font>
<br> MA1 Mechanical Engineering <br>  __Mechanical vibrations__ <br> <br> 
***
***
<font color='Navy'><h1><center>Session 3 : Multiple DOF system</center></h1></font> 
***
 <div class="alert alert-block alert-info">
<center><b>Welcome to the third session </b></center></div>
    
During the practical session, we will use the same libraries as the ones used in the previous sessions.

<div class="alert alert-block alert-info">
<center><b>Don't forget :It is important that you run each cell of the notebook. To do so select a cell (it will be highlighted) and press shift+enter on your keyboard or the play button in the menu above.</b></center></div>

In [ ]:
# let's import both numpy and matplotlib.pyplot 
# the "as" in the code bellow is to tell python that if we reference to the package using np and plt 
import numpy as np
import matplotlib.pyplot as plt 
import scipy.linalg as linalg
# don't worry about this line 
%matplotlib notebook

# Exercises

## Exercise 1: Fundamental equations of a MDOF system

Consider the following three-degree-of-freedom (3 DOFs) system :
<img src="./Images/MDOF.png" width=200 height=200 />


<b> Assignment 1.1:</b> Write the equation of motion in the time domain for the system (in LaTeX)  in the cell below: 

(If you are not familiar with LaTeX you can use: https://www.hostmath.com/ and copy the LaTeX below, or write it on paper for yourself)

###BEGIN SOLUTION
$$
M {\bf \ddot  x}(t) + C {\bf\dot x}(t) + K {\bf x}(t) = {\bf f}(t) \text{ ,with: } M =  \begin{bmatrix} m & 0 & 0\\ 0&m&0\\0&0&m \end{bmatrix} ,  C =  \begin{bmatrix} c1+c & -c & 0\\ -c&2c&-c\\0&-c&c \end{bmatrix}  ,  K =  \begin{bmatrix} k1+k & -k & 0\\ -k&2k&-k\\0&-k&k \end{bmatrix} .
$$
$$
{\bf f}(t) = \begin{Bmatrix} 0 \\ 0 \\ f(t) \end{Bmatrix}
$$
$$
{\bf x}(t) = \begin{Bmatrix} x_1(t) \\ x_2(t) \\ x_3(t) \end{Bmatrix}
$$
###END SOLUTION

<b> Assignment 1.2:</b> Create a function `build_3DOF_matrices` that takes as arguments the stiffnesses $k$ and $k_1$, the damping values $c$ and $c_1$ and the mass $m$ and computes the system matrices ($M$, $C$, $K$). In the cell below we have given you a start, we show how you can use `np.array` to build the $M$-matrix. 

In [ ]:
def build_3DOF_matrices(m:float,c:float,c1:float,k:float,k1:float): 
    """
    Implement a function that computes the build the system matrix for a 3DOF matrix

    Arguments:
    m,k,k1,c,c1 -- parameters of the MDOF system

    Returns:
    M_mat : The M matrix from the system equation
    K_mat : The K matrix from the system equation
    C_mat : The C matrix from the system equation
  
    Tips use np.array([[],[],[]]) to define your 2D matrix
    check documentation FMI 'Numpy.array()'
    """  
    # here is an example of how to set a matrix 
    
    M_mat=np.array([[m, 0, 0],
           [0, m, 0],
           [0, 0, m]])
    
    ###BEGIN SOLUTION
    
    K_mat=np.array([[k1+k, -k, 0],
           [-k, 2*k, -k],
           [0, -k, k]])
    C_mat=np.array([[c1+c, -c, 0],
            [-c, 2*c, -c],
            [0,-c,c]])
    ###END SOLUTION
    return M_mat, C_mat, K_mat

In the cell below we will check if you implemented the function correctly. To do so we will use several properties of the system matrices.  Do you understand all the checks that are being done?

In [ ]:
M, C, K = build_3DOF_matrices(m=1, k=10, k1=10, c=0.1, c1=0.1)

assert np.array_equal(M, np.eye(3)), 'M-matrix is not diagonal' # Mass matrix is a diagonal matrix

for matrix in [M,C,K]: # Iterate over the three matrices
    assert np.size(matrix,0) == np.size(matrix, 1), f'{matrix} is not square' # All three matrices should be square
    assert np.size(matrix,0) == 3, f'{matrix} is not 3x3' # For a 3DOF system we expect the system matrices to be 3X3 matrix
    assert np.allclose(matrix, matrix.T), f'{matrix} is not symmetric' # The matrices should be symmetric
    # The next one is a bit special, we assert whether or not the matrix is positive definite
    assert np.all(np.linalg.eigvals(matrix) > 0), f'{matrix} is not positive definite' 
# Some final checks
assert K[2,2] == 10
assert np.sum(K) == 10,  'The K matrix has an error in it'
assert np.sum(C) == 0.1, 'The C matrix has an error in it'


## Exercise 2 : Solving the conservative system

Before diving into a damped response we first will analyze the __conservative__ system response. Let us start with the eigenvalues and eigenvectors of the conservative system. To do so, we need to solve the following equation:

$(K-\lambda{}M)\Psi = (K-\omega^2M)\Psi =0$

Which is a basic eigenvalue problem. In the cell below we provide you a function `calc_eigenvalues_eigenvectors` that uses `scipy.linalg.eig` to solve the eigenvalue problem and returns $\lambda$ and $\Psi$.


In [ ]:
def calc_eigenvalues_eigenvectors(M:np.array,K:np.array):
    """
    A function that computes the eigenvalues and eigenvectors
    based on the system parameter by reworking the equations of motion to a generalized eigenvalue problem

    Arguments:
    M,K -- system matrices of the MDOF system

    Returns:
    lamb -- The eigenvalues of the conservative eigenvalue problem
    psi -- The eigenvectors of the conservative eigenvalue problem
  
    
    """   
    
    lamb, psi = linalg.eig(K,M)

    return lamb, psi
    

<b> Assignment 2.1:</b> Use `build_3DOF_matrices` and `calc_eigenvalues_eigenvectors`  to first calculate the eigenvalues `lamb` and eigenvectors `psi` (a.k.a the mode shapes).  Then calculate the eigenfrequencies in $Hz$ for a 3DOF system as presented above with $m = 1 kg$, $k = k1 = 16 N/m$, $c = c1 = 0.1 N s/m$.

Put your found eigenfrequencies in an array called `f_res`.

In [ ]:
###BEGIN SOLUTION

M, C, K = build_3DOF_matrices(m=1, k=16, k1=16, c=0.1, c1=0.1)
lamb, psi = calc_eigenvalues_eigenvectors(M,K)

f_res = np.sqrt(lamb)/2/np.pi

###END SOLUTION

Let us have a look at the resonance frequencies you calculated:

In [ ]:
# Enumerate() method adds a counter to the list f_res and returns it in a form of elements_index and elements_values
for i,f_r in enumerate(f_res):
    print(f'Resonance frequency {i}: {np.real(f_r):0.2f} Hz')

# I do a little test, summing up the found resonance frequencies and comparing to the correct value
assert len(f_res) == 3, f'Your result contains {len(f_res)} resonance frequencies, we have 3 (pairs of) resonance frequencies'
assert round(np.sum(np.real(f_res)),2) != 12.73, 'Did you spot the power two that applies to omega?'
assert round(np.sum(np.real(f_res)),2) == 2.22, 'At least one of the resonance frequencies you found is incorrect!'

<b> Assignment 2.2:</b> In the field below answer following questions: 
* How many resonance frequencies did you obtain? Why?
* How many mode shapes did you obtain? Why?
* What are the dimensions of the mode shapes? Why?

###BEGIN SOLUTION

While we will obtain 6 results, the results are 3 complex conjugate pairs (= the number of Degrees of Freedom, DOFs). Meaning we in fact find:

- 3 resonance frequencies, as we have three Degrees of Freedom (DOFs)
- 3 mode shapes are obtained, as many as there are resonances frequencies/modes
- the mode shapes are vectors of dimensions 3x1, as we have 3 dof  

###END SOLUTION

An important property of mode shapes is their orthogonality. In the next assignment you have to validate the orthogonality between two mode shapes.

Remember that :

$\Psi^TM\Psi=diag(\mu_i)$,

$\Psi^TK\Psi=diag(\mu_i\omega_i^2)$


<b> Assignment 2.3:</b> Complete `check_orthogonality` to calculate the orthogonality between two vectors. Consider using `np.matmul` and `np.transpose`: 

In [ ]:
def check_orthogonality(phi_1, phi_2, M):
    """
    A function that computes the orthogonality between two vectors

    Arguments:
    psi_1, psi_2 -- vectors of which the orthogonality is checked
    M    -- Mass matrix of the system

    Return: orth -- the orthogonality of the two vectors 
    
    TIP: matrix multiplications are done using np.matmul
    """   
    ###BEGIN SOLUTION
    
    orth_left = np.matmul(np.transpose(phi_1), M) 
    orth = np.matmul(orth_left, phi_2)
    ###END SOLUTION

    return orth

Let us check whether your function does what it must do:

In [ ]:
# As a test let us take two orthogonal vectors
vec_1 = np.array([1,1,0])
vec_2 = np.array([1,-1,0])
M_test = np.eye(3) # The unity matrix

assert np.abs(check_orthogonality(vec_1, 5*vec_1, M_test)) > 1.e-10 # not equal to zero, a vector is not orthogonal to itself
assert np.isclose(check_orthogonality(vec_1, vec_2, M_test), 0) # = zero up to numerical precision
assert np.isclose(check_orthogonality(vec_1, 5*vec_2, M_test), 0) # = zero up to numerical precision

<b> Assignment 2.4:</b> Use `check_orthogonality` to check the orthogonality between your modeshapes:


In [ ]:
###BEGIN SOLUTION

results = np.empty((3,3))
i = 0
j = 0

for psi_i in psi.T:
    for psi_j in psi.T:
        results[i,j] = round(np.abs(check_orthogonality(psi_i, psi_j, M=M)),2)
        j+=1
    i+=1
    j=0

results
###END SOLUTION

Were your mode shapes nicely orthogonal?

## Exercise 3 : The frequency response function

Let us continue with the 3DOF freedom model we used in Exercise 1. This time we will focus on the frequency response function. The frequency response function is a way to represent how a system will respond to a force being exerted on any of the degrees of freedom. For this we start with standard equations of motion of a MDOF system expressed in the frequency domain.

$\left(-\omega^2M+j\omega{}C+K\right)\vec{X}(\omega) = \vec{F}(\omega)$

Which we can rewrite as follows:

$\vec{X}(\omega) = \left(-\omega^2M+j\omega{}C+K\right)^{-1}\vec{F}(\omega) = H(\omega)\vec{F}(\omega)$

In which we call $H(\omega)$ the frequency response function.

Which we can rewrite as follows:

$\vec{X}(\omega) = \left(-\omega^2M+j\omega{}C+K\right)^{-1}\vec{F}(\omega) = H(\omega)\vec{F}(\omega)$

In which we call $H(\omega)$ the frequency response function.

In the cell below the function `frequency_response_function` calculates the FRF for a system defined using $M$, $C$ and $K$ matrices for a range of frequencies.

In [ ]:
def frequency_response_function(M:np.array, C:np.array, K:np.array, f= np.linspace(0,4,100), i=1,j=1):
    """
    A function that computes the Frequency Response Function from the system matrices

    Arguments:
    M,C,K -- system matrices of the MDOF system
    f -- the frequency for which the repsponse needs to be calculated
    i,j -- the specific element of the FRF matrix to be computed.

    Return: H_ij -- the i-th row and j-th column element of the FRF function at f
    """
    
    if not isinstance(f, np.ndarray):
        f = np.array([f])
    s = f*np.pi*2*1j
    
    M_s2 = np.outer(M,(s**2)).reshape(M.shape[0], -1, len(s))
    C_s = np.outer(C,(s**1)).reshape(M.shape[0], -1, len(s))
    K_s = np.outer(K,(s**0)).reshape(M.shape[0], -1, len(s))

    H_ij = []
    for s_i in range(len(s)):
        H_ij.append(np.linalg.inv((M_s2+C_s+K_s)[:,:,s_i])[i,j])
    
    return H_ij

<b> Assignment 3.1:</b> In the field below answer following questions:

* What are the dimensions of the FRF for a 3DOF system?
* Can you explain in your own words what the FRF represents?

###BEGIN SOLUTION
- The FRF of 3DOF system is 3x3 matrix 
- The elements ij of the FRF (i = row, j= columns) gives the frequency responce of the dof i if the excitation were applied in the dof j

###END SOLUTION

<b> Assignment 3.2:</b> For the 3DOF system of exercise 1, can you calculate the transfer function between the input location of the force $f$ and the output location $x_3$ for a range of frequencies between 0 and 2Hz using `frequency_response_function`? Call your frequency vector `f` and your output `H_3f`
    
**TIP**: Be aware that in python counting starts from 0, so the first column of a matrix is denoted 0



In [ ]:
###BEGIN SOLUTION
f= np.linspace(0,2,200)
M, C, K = build_3DOF_matrices(m=1, k=16, k1=16, c=0.1, c1=0.1)
H_3f = frequency_response_function(M, C, K, f= f, i=2,j=2)
###END SOLUTION

Let us just quickly check if you have done the right steps.

In [ ]:
assert max(f) == 2 # Maximum frequency is 2Hz
assert len(H_3f) == len(f)

### BEGIN HIDDEN TESTS
M_t, C_t, K_t = build_3DOF_matrices(m=1, k=16, k1=16, c=0.1, c1=0.1)
H_3f_t = frequency_response_function(M, C, K, f= f, i=2,j=2)

assert np.array_equal(M_t, M), 'There is an error in the M matrix'
assert np.array_equal(C_t, C), 'There is an error in the C matrix'
assert np.array_equal(K_t, K), 'There is an error in the K matrix'

assert np.array_equal(H_3f_t, H_3f), 'There is an error in the Transfer function, did you calculate the right transfer function?'
### END HIDDEN TESTS




And let us have a look at your output:

In [ ]:
assert ran_test_cell

fig, axes = plt.subplots(2,1)

axes[0].semilogy(f,np.abs(H_3f))
axes[0].grid(which='both', linestyle=':')
axes[0].set_ylabel('Magnitude')
axes[1].plot(f,np.angle(H_3f,deg=True))
axes[1].set_ylabel('Angle (°)')
axes[1].set_xlabel('Frequency (Hz)')

axes[1].grid(which='both', linestyle=':')

t = axes[0].set_title('H_33(f)')

Obviously the properties of the FRF will change depending on the system properties. In the following interactive cell you can change the values of $m$, $c$, $k$. Can you identify the role of each of the parameters on the final FRF?

In [ ]:
import iplot.interactive_plot_1

As discussed in assignment 3.1 the FRF is in fact a 3 by 3 matrix for a three dimensional system. Up to this point we just focused on the transfer function between input location $f$ and output location $x_3(t)$. But how do the other elements of the transfer function look like?

<b> Assignment 3.3:</b> Update the code below to plot the FRF function for all 9 elements. Can you also plot a vertical line at the resonance frequencies on the x-axis? 
    
**TIP** : To plot a figure with logarithmic y-scale on a particular axis `ax` you can use the command `ax.semilogy` and to draw a single vertical line the same axis you can use the command `ax.vline(value)`.


In [ ]:
f = np.linspace(0,2,200)
n_dof = np.shape(M)[0]

fig, axes = plt.subplots(n_dof,n_dof)

for i in range(n_dof):
    for j in range(n_dof):
        # V---!ADD THE NECESSARY CODE HERE!---V
        ###BEGIN SOLUTION
        axes[i,j].semilogy(f, np.abs(frequency_response_function(M,C,K,f=f, i=i,j=j)))
        for fr in f_res:
            axes[i,j].axvline(np.real(fr), linestyle=':', color='r')
        ###END SOLUTION
        axes[i,j].set_title(f'$||H_{{{i+1}{j+1}}}||$')
        axes[i,j].set_xlabel('Frequency (Hz)')
        axes[i,j].grid(which='both', linestyle=':')
        axes[i,j].set_ylim([1e-3,1e1])
        axes[i,j].set_xlim([0, max(f)])
        
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.9)

<b> Assignment 3.4:</b> Can you answer following questions about the FRF's in the cell below?

* Do you notice any difference between the FRF's on the diagonal elements and those on non-diagonal elements?
* What do you notice about the resonance frequencies?
* What can you tell about the zeroes of the system?
* Can you spot other properties of the frequency response function?

###BEGIN SOLUTION

- In FRF's on the diagonal the resonances always are alternated by an anti-resonance, off-diagonal FRF's don't have this property
- They are visible at the same frequencies in all FRFs
- They are not situated at the same frequency for each element
- The matrix is symmetric

###END SOLUTION

<div class="alert alert-block alert-success">
<b>That concludes this session:</b> In this session we tried to show you how MDOF systems can be modelled and how the FRF functions can be computed from the system matrices.
</div>
